In [ ]:
import pandas as pd
import altair as alt
import json

alt.renderers.enable('default')

df = pd.read_csv('../datas/dpt2020.csv', sep=';')
df_male = df[df['sexe'] == 1]

df_top = df_male.sort_values(['dpt', 'annais', 'nombre'], ascending=[True, True, False])
df_top = df_top.groupby(['dpt', 'annais']).first().reset_index()

def count_name_changes(group):
    return group['preusuel'].ne(group['preusuel'].shift()).sum()

name_changes = df_top.groupby('dpt', group_keys=False).apply(count_name_changes).reset_index(name='changes')
years = df_top.groupby('dpt')['annais'].nunique().reset_index(name='n_years')
instability = pd.merge(name_changes, years, on='dpt')
instability['instability'] = instability['changes'] / instability['n_years']

with open('../datas/departements.geojson', 'r') as f:
    france_geo = json.load(f)

chart = alt.Chart(alt.Data(values=france_geo['features'])).mark_geoshape(
    stroke='white'
).encode(
    color=alt.Color('instability:Q', scale=alt.Scale(scheme='oranges'), title='Instabilité'),
    tooltip=[
        alt.Tooltip('properties.nom:N', title='Département'),
        alt.Tooltip('instability:Q', format=".2f", title='Instabilité')
    ]
).transform_lookup(
    lookup='properties.code',
    from_=alt.LookupData(instability, 'dpt', ['instability'])
).project(
    type='mercator'
).properties(
    width=800,
    height=800,
    title='Instabilité des prénoms masculins en tête par département'
)

chart